# 什么是卷积神经网络
- 卷积神经网络就是用一个小矩阵（卷积核）在原来图片像素矩阵上平移，做矩阵叉乘，然后生成一个新的矩阵
- 这个时候如果原始图片中的矩阵与卷积核比较相似，那么做完矩阵运算之后，得到的值就会比较大，如果不相似，那么值就会比较小。
- 然后通过激活函数，相似的地方就会被机会，不相似的地方就会被忽略

# 为什么要使用卷积神经网络
- 就图像处理来说，图像本来是一个二维的点矩阵，如果不使用卷积神经网络，我们就不得不把这些点矩阵拉伸成一个一维的向量，然后在喂给MLP模型，这个时候会发生参数爆炸
- 以12M像素的彩色图片为例，他总共有36000000个像素点，如果我们做一个隐藏层的MLP，假设这个隐藏层上有100个神经元，那么这个神经网络就会有36000000*100=3.6B的参数，这个需要14GB的内存，这时候这个矮胖的模型基本上就没有办法训练了
- 另一个重要的原因在于将二维矩阵拉伸成一个一维的向量，原来图片上的一些信息会丢失掉

# 为什么卷积神经网络是有效的
- 平移不变形：也就是说不管你在图片上如何移动目标小区域，都不会影响卷积神经网络在这张图片上的效果
- 局部性：只让神经网络注意图像中的局部区域，而不必在意图像中相隔较远的区域之间的关系

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
# 模拟卷积的运算过程，做一个二维相关的运算
def corr2d(X,K):
    # 代表卷积核的高度和宽度
    h,w = K.shape
    # 初始化卷积运算之后的结果
    Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    # 模拟卷积核在原始图片像素矩阵上开始平移计算
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y

In [4]:
# 演示一下卷积的运算过程
X = torch.arange(16).view((4,4))
K = torch.tensor([[1,0,1],[0,1,0],[1,0,1]])
Y = corr2d(X,K)
print(Y)

tensor([[25., 30.],
        [45., 50.]])


In [5]:
# 用上面的卷积函数是一个二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size) -> None:
        super().__init__()
        self.weight = nn.Parameter(kernel_size)
        # 每次卷积核做完运算之后，都会加上一个偏置项
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, X):
        return corr2d(X,self.weight) + self.bias

In [11]:
# 一个用卷积层实现的图像边缘检测
# 其中1代表黑色，0代表白色
X = torch.ones(6,8)
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [12]:
# 初始化这个卷积核
# 如果图片上的候选区域像素没有发生改变，那么这个区域与卷积核矩阵叉乘之后的结果是0
# 如果图片上的候选区域像素由黑变成白，那么这个区域与卷积核矩阵叉乘之后的结果是1
# 如果图片上的候选区域像素由白变成黑，那么这个区域与卷积核矩阵叉乘之后的结果是-1
K = torch.tensor([[1, -1]])
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

# 图像检测的结果
- 在第二列的时候发生了黑变白的过程
- 在原矩阵倒数第三列的时候发生了白变黑的过程（经过卷积核叉乘后，矩阵大小变成X.shape-K.shape+1）

In [8]:
# 上面那个边缘检测是无法检测出来水平变换的
Y = corr2d(X.T, K)
Y

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

# 学习由X生成Y的卷积核

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2*conv2d.weight.grad
    if(i+1)%2 == 0:
        print(i+1, l.sum().item())

2 0.9026836156845093
4 0.1517886072397232
6 0.025606578215956688
8 0.004353695083409548
10 0.0007540329825133085


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


In [14]:
conv2d.weight.data

tensor([[[[ 0.9957, -0.9944]]]])